In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q transformers==4.34.0 datasets==2.14.5 accelerate==0.23.0 evaluate==0.4.1 peft==0.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.2 MB/s eta 0:00:00


##Train MedMCQA Dataset using `Flan-T5`

In [3]:
%cd /content/drive/MyDrive/Project/MCQA/MedMCQA

/content/drive/MyDrive/Project/MCQA/MedMCQA


###**1. Load Data**

In [4]:
from datasets import load_dataset
from datasets import DatasetDict

data_dir = "./data"

raw_dataset = {
    'train': load_dataset('json', data_files=f'{data_dir}/train.json')['train'],
    'valid': load_dataset('json', data_files=f'{data_dir}/dev.json')['train']
}
raw_dataset = DatasetDict(raw_dataset)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

###**2. Dataloader**

In [5]:
import torch

id2label = {
    0: 'A',
    1: 'B',
    2: 'C',
    3: 'D',
    4: 'E'
}

def preprocess_function(examples, max_seq_length, tokenizer):
    # Tokenize the texts
    sentences = []
    labels = []
    for example in zip(examples["question"], examples["exp"],
                       examples['opa'], examples['opb'], examples['opc'], examples['opd'],
                       examples['cop']):
        question = example[0]
        context = example[1]
        opa = example[2]
        opb = example[3]
        opc = example[4]
        opd = example[5]
        choices = f"A{opa}. \n B. {opb}. \n C. {opc} \n D. {opd}"
        prompt = f"Context: {context}. Question: {question}. Choice the correct answers from: {choices}"
        sentences.append(prompt)

        answer = id2label[int(example[6]-1)]
        labels.append(answer)


    model_inputs = tokenizer(
        sentences,
        padding="max_length",
        max_length=max_seq_length,
        truncation=True
    )
    labels = tokenizer(
        labels,
        padding=True
    )
    model_inputs["labels"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    ]
    return model_inputs

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import get_peft_model, LoraConfig, TaskType

model_name = 'google/flan-t5-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# peft_config = LoraConfig(
#         r=8,
#         lora_alpha=32,
#         lora_dropout=0.05,
#         target_modules=["query", "vavlue"],
#         bias="none"
#     )

# model = get_peft_model(model, peft_config)

In [8]:
from functools import partial

processed_dataset = raw_dataset.map(
    partial(
        preprocess_function,
        max_seq_length=256,
        tokenizer=tokenizer
    ),
    batched=True,
    load_from_cache_file=False,
    remove_columns=['question', 'exp', 'cop', 'opa', 'opb', 'opc', 'opd', 'subject_name', 'topic_name', 'id', 'choice_type'],
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/182822 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/4183 [00:00<?, ? examples/s]

In [9]:
processed_dataset['train']

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 182822
})

###**3. Metric**

In [10]:
import numpy as np
import evaluate
from transformers import EvalPrediction

label2id = {
    'A': '0',
    'B': '1',
    'C': '2',
    'D': '3',
    'E': '4'
}

def postprocess_text(predictions, labels):
    predictions = [prediction.strip() for prediction in predictions]
    labels = [label2id[label.strip()] for label in labels]

    for idx in range(len(predictions)):
        if predictions[idx] in label2id:
           predictions[idx] = label2id[predictions[idx]]
        else:
            predictions[idx] = '-100'
    return predictions, labels

def load_metric(metric_name):
    if metric_name == "accuracy":
        return evaluate.load("accuracy")
    elif metric_name == "f1":
        return evaluate.load("f1")

def seq2seq_compute_metrics(tokenizer, metric):
    def compute_metrics(eval_pred: EvalPrediction):
        nonlocal tokenizer, metric
        predictions, labels = eval_pred
        if isinstance(predictions, tuple):
            predictions = predictions[0]

        predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        predictions, labels = postprocess_text(predictions, labels)
        result = metric.compute(predictions=predictions, references=labels)
        return result
    return compute_metrics

In [11]:
metric = load_metric("accuracy")
compute_metrics = seq2seq_compute_metrics(tokenizer, metric)

###**4.Trainer**

In [12]:
import transformers
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

In [13]:
label_pad_token_id = -100

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [14]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f'./model/{model_name}',
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    evaluation_strategy='steps',
    save_strategy='steps',
    save_steps=2000,
    eval_steps=2000,
    save_total_limit=2,
    predict_with_generate=True,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy'
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    train_dataset=processed_dataset['train'],
    eval_dataset=processed_dataset['valid']
)

In [15]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
2000,0.311100,0.433079,0.609849
4000,0.298200,0.426535,0.610328
6000,0.284200,0.427150,0.617499
8000,0.286600,0.417515,0.622042
10000,0.281500,0.419718,0.618217
12000,0.261100,0.415310,0.622759
14000,0.259400,0.417608,0.617499
16000,0.259900,0.414758,0.627301
18000,0.258500,0.412855,0.625628
20000,0.252000,0.411286,0.626823


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=22854, training_loss=0.27823444158477795, metrics={'train_runtime': 8077.2713, 'train_samples_per_second': 45.268, 'train_steps_per_second': 2.829, 'total_flos': 1.2518870868924826e+17, 'train_loss': 0.27823444158477795, 'epoch': 2.0})

##Predict ViMedMCQA

In [16]:
%cd /content/drive/MyDrive/Project/MCQA/ViMedicalMCQA

/content/drive/MyDrive/Project/MCQA/ViMedicalMCQA


In [17]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [18]:
import pandas as pd
df = pd.read_csv("./data/public_test_with_context_en.csv")

In [19]:
convert_to_binary = {
    'A': '10000',
    'B': '01000',
    'C': '00100',
    'D': '00010',
    'E': '00001'
}

def predict_per_sample(df, model, tokenizer):
    answers = []
    for idx, row in df.iterrows():
        context = row["context"].split("///")[0]
        question = row["question"]
        choices = [
            op for op in row[["option_1", "option_2", "option_3", "option_4", "option_5", "option_6"]].tolist()
            if isinstance(op, str)
        ]
        text_choices = " ".join(choices)
        prompt = f"Context: {context}. Question: {question}. Choice the correct answers from: {text_choices}"
        model_inputs = tokenizer(prompt, return_tensors="pt")
        outputs = model.generate(**model_inputs)
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
        if prediction in convert_to_binary:
            answer = convert_to_binary[prediction]
        else:
            answer = '00000'
        answer = answer[:len(choices)]
        answers.append(answer)
    return answers

In [25]:
model = model.to("cpu")

In [ ]:
answers = predict_per_sample(df, model, tokenizer)

In [27]:
answers

['0100',
 '0100',
 '0010',
 '00',
 '1000',
 '0100',
 '0010',
 '00',
 '000',
 '00',
 '00100',
 '010',
 '001',
 '001',
 '100',
 '001',
 '0010',
 '0100',
 '100',
 '000',
 '000',
 '001',
 '000',
 '100',
 '0100',
 '00',
 '001',
 '010',
 '0100',
 '0100',
 '0100',
 '0100',
 '000',
 '000',
 '100',
 '001',
 '0100',
 '00',
 '001',
 '00',
 '001',
 '010',
 '010',
 '0100',
 '01000',
 '010',
 '010',
 '00',
 '001',
 '010',
 '01',
 '100',
 '100',
 '010',
 '100',
 '1000',
 '00',
 '010',
 '100',
 '000',
 '000',
 '1000',
 '100',
 '0001',
 '100',
 '000',
 '000',
 '010',
 '1000',
 '1000',
 '000',
 '100',
 '000',
 '0010',
 '000',
 '000',
 '1000',
 '0010',
 '0010',
 '010',
 '10000',
 '001',
 '0100',
 '1000',
 '000',
 '10000',
 '001',
 '0001',
 '000',
 '000',
 '0001',
 '0001',
 '0001',
 '0001',
 '0001',
 '1000',
 '0001',
 '0001',
 '0100',
 '000']

In [28]:
df['answer'] = answers

In [29]:
result_df = df[["id", "answer"]]

In [30]:
result_df

,id,answer
0,level3_1,0100
1,level3_2,0100
2,level3_5,0010
3,level3_13,00
4,level3_14,1000
...,...,...
95,level4_4,1000
96,level4_9,0001
97,level4_27,0001
98,level4_28,0100


In [32]:
result_df.to_csv('./data/result_en_flan_t5_base.csv', index=False)